In [3]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import fsolve
from scipy.stats import multivariate_normal

In [2]:
pd1 = 0.1
pd2 = 0.2
pd3 = 0.3
pd4 = 0.4
rho12 = 0.1
rho13 = 0.2
rho14 = 0.3
rho23 = 0.4
rho24 = 0.5
rho34 = 0.6

In [18]:
#Draw values of the correlated latent variables 𝒁i from a multivariate normal distribution with mean vector 𝜇 = (0, 0, 0, 0) and covariance matrix 𝜎2 = (1, 𝜌12, 𝜌13, 𝜌14; 𝜌12, 1, 𝜌23, 𝜌24; 𝜌13, 𝜌23, 1, 𝜌34; 𝜌14, 𝜌24, 𝜌34, 1) using multivariate_normal.rvs() function.

z1 = multivariate_normal.rvs(mean=[0,0,0,0], cov=[[1,rho12,rho13,rho14],[rho12,1,rho23,rho24],[rho13,rho23,1,rho34],[rho14,rho24,rho34,1]], size=1000000)
invphipd1 = stats.norm.ppf(pd1)
invphipd2 = stats.norm.ppf(pd2)
invphipd3 = stats.norm.ppf(pd3)
invphipd4 = stats.norm.ppf(pd4)

In [17]:
print(invphipd1)
print(invphipd2)
print(invphipd3)
print(invphipd4)

-1.2815515655446004
-0.8416212335729142
-0.5244005127080409
-0.2533471031357997
